In [ ]:
"""
Purpose: 
To show how to query the morphology of a processed neuron

"""

# Importing Relevant Packages

In [1]:
from os import sys
sys.path.append("/meshAfterParty/")

In [2]:

import neuron
import neuron_visualizations as nviz
import neuron_utils as nru
from pathlib import Path

# Load a Cell to analyze

In [3]:
excitatory_file = "95442489112204099_excitatory_7"

recovered_neuron = nru.decompress_neuron(filepath=f"../test_neurons/spine_detection/excitatory_meshafterparty/{excitatory_file}_meshafterparty",
                     original_mesh=f"../test_neurons/spine_detection/{excitatory_file}",
                     )

Decompressing Neuron in minimal output mode...please wait


In [12]:
nviz.visualize_neuron(recovered_neuron,
                      visualize_type=["mesh","skeleton"],
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton


# What the Querying Functions Look like

In [ ]:
"""
--- Excerpt from neuron_searching.py


@run_options(run_type="Branch")
def median_mesh_center(branch,limb_name=None,branch_name=None,**kwargs):
    return branch.width_new["median_mesh_center"]

@run_options(run_type="Branch")
def no_spine_mean_mesh_center(branch,limb_name=None,branch_name=None,**kwargs):
    return branch.width_new["no_spine_mean_mesh_center"]

@run_options(run_type="Branch")
def no_spine_median_mesh_center(branch,limb_name=None,branch_name=None,**kwargs):
    return branch.width_new["no_spine_median_mesh_center"]
    
....


@run_options(run_type="Limb")
def skeleton_distance_limb(curr_limb,limb_name=None,**kwargs):
    curr_skeleton = curr_limb.get_skeleton()
    return sk.calculate_skeleton_distance(curr_skeleton)

@run_options(run_type="Limb")
def n_faces_limb(curr_limb,limb_name=None,**kwargs):
    return len(curr_limb.mesh.faces)

@run_options(run_type="Limb")
def merge_limbs(curr_limb,limb_name=None,**kwargs):
    return "MergeError" in curr_limb.labels

@run_options(run_type="Limb")
def limb_error_branches(curr_limb,limb_name=None,**kwargs):
    error_nodes = nru.classify_endpoint_error_branches_from_limb_concept_network(curr_limb.concept_network)
    node_names = np.array(list(curr_limb.concept_network.nodes()))
    return dict([(k,k in error_nodes) for k in node_names])

@run_options(run_type="Limb")
def average_branch_length(curr_limb,limb_name=None,**kwargs):
    return np.mean([sk.calculate_skeleton_distance(curr_limb.concept_network.nodes[k]["data"].skeleton) for k in curr_limb.concept_network.nodes()])


"""

In [17]:
import skeleton_utils as sk
import neuron_searching as ns


# def some_function(curr_limb):
#     ...
#     return stat
functions_list=[
ns.n_faces_branch,
"width",
ns.skeleton_distance_branch,
ns.skeleton_distance_limb,
    
"n_faces_limb",
ns.limb_error_branches,
ns.average_branch_length
    

]

local_dict = dict()

#Example Query 1:
current_query = "(average_branch_length > 10000) and width < 500"

# dataframe_result = ns.query_neuron(recovered_neuron,
#                          functions_list,
#                           current_query,
#                           query_variables_dict=local_dict,
#                           return_dataframe=True,
#                           return_limbs=False,
#                           return_limb_grouped_branches=True,
#                          print_flag=False)

# dataframe_result


In [14]:
returned_query = ns.query_neuron(recovered_neuron,
                         functions_list,
                          current_query,
                          query_variables_dict=local_dict,
                          return_dataframe=False,
                          return_limbs=False,
                          return_limb_grouped_branches=True,
                         print_flag=False)

returned_query

{'L0': array([ 4,  8,  9, 10, 15, 22, 31, 33, 35, 37, 41, 44, 46]),
 'L1': array([3]),
 'L2': array([ 2,  5,  9, 10]),
 'L3': array([1])}

In [16]:
nviz.visualize_neuron(recovered_neuron,
                     limb_branch_dict=returned_query,
                      mesh_color="red",
                      mesh_color_alpha=1,
                     mesh_whole_neuron=True)


 Working on visualization type: mesh


# Example Query 2:

In [8]:


current_query = "n_faces_branch in @list_of_faces or skeleton_distance_branch > @branch_threshold or width > 10000"

local_dict = dict(list_of_faces=[100,200,300,400],
          branch_threshold=400)

returned_query = ns.query_neuron(recovered_neuron,
                         functions_list,
                          current_query,
                          query_variables_dict=local_dict,
                          return_dataframe=False,
                          return_limbs=False,
                          return_limb_grouped_branches=True,
                         print_flag=False)

returned_query

{'L0': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]),
 'L1': array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 'L2': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 'L3': array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 'L4': array([0, 1, 2, 3, 4]),
 'L5': array([0])}

In [9]:
nviz.visualize_neuron(recovered_neuron,
                     limb_branch_dict=returned_query,
                      mesh_color="red",
                     mesh_whole_neuron=True)


 Working on visualization type: mesh


# Finding Axon Like Parts 

In [20]:
axon_width_like_requirement = "(median_mesh_center < 400)"# or no_spine_median_mesh_center < 150)"
axon_width_like_query = (f"(n_spines < 4 and {axon_width_like_requirement} and skeleton_distance_branch <= 15000)"
                f" or (skeleton_distance_branch > 15000 and {axon_width_like_requirement} and spines_per_skeletal_length < 0.00023)")

axon_width_like_functions_list = [
    "width",
    "median_mesh_center",
    "n_spines",
    "n_faces_branch",
    "skeleton_distance_branch",
    "spines_per_skeletal_length",
    "no_spine_median_mesh_center",
]

axon_like_limb_branch_dict = ns.query_neuron(recovered_neuron,
                                       query=axon_width_like_query,
                   functions_list=axon_width_like_functions_list)

axon_like_limb_branch_dict

{'L0': array([ 0,  4,  8,  9, 10, 12, 13, 14, 15, 22, 31, 33, 35, 36, 37, 41, 44,
        45, 46, 47, 48]),
 'L1': array([0, 1, 3, 7]),
 'L2': array([ 1,  2,  5,  7,  9, 10]),
 'L3': array([1]),
 'L4': array([0, 3, 4]),
 'L5': array([0])}

In [21]:
nviz.visualize_neuron(recovered_neuron,
                     limb_branch_dict=axon_like_limb_branch_dict,
                      mesh_color="red",
                      mesh_color_alpha=1,
                     mesh_whole_neuron=True)


 Working on visualization type: mesh


In [27]:
sub_mesh = []
for l_idx,branch_list in axon_like_limb_branch_dict.items():
    branch_meshes = [recovered_neuron[l_idx][k].mesh for k in branch_list]
    sub_mesh += branch_meshes
import trimesh_utils as tu

In [30]:
total_subtract = tu.combine_meshes(sub_mesh)
nviz.plot_objects(total_subtract,mesh_alpha=1)